## Atomic bomb tests from 1945 - 2009

Import the Atomic Bombs dataset from `github` repository into a `pandas` dataframe. Note this reads in the `.csv` version of the dataset directly from `github`.

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# read-in the dataset from github into a pandas dataframe
url = "https://raw.githubusercontent.com/xbr209/tutorials/master/Atomic_bombs.csv"
dataset = pd.read_csv(url)

# print out the pandas dataframe to screen
dataset

Define function to create animation of atomic bomb tests, with points coloured by country and sized by bomb yield. 

In [ ]:
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode()

def animation(dataset, countries, years):
    # define location of world map image for background under (scatter plot) animation
    source_image_url = "https://raw.githubusercontent.com/xbr209/tutorials/master/world_map.png"
    image_layout = go.layout.Image(source=source_image_url, xref= "x", yref= "y", x= -180, y= 90, sizex= 360, \
                    sizey= 180, sizing= "stretch", opacity= 0.75, layer= "below")

    # define figure contents using a dictionary
    fig_dict = {
        "data": [],
        "layout": {},
        "frames": []
    }

    # fill in most of the layout
    fig_dict["layout"]["yaxis"] = {"range": [-90, 90], "showgrid": False, "zeroline": False, "showticklabels": False}
    fig_dict["layout"]["xaxis"] = {"range": [-180, 180], "showgrid": False, "zeroline": False, \
                    "showticklabels": False, "scaleanchor": "y", "scaleratio": 1, "titlefont": {"size": 1}, \
                    "tickfont": {"size": 1}}
    fig_dict["layout"]["hovermode"] = "closest"
    fig_dict["layout"]["sliders"] = {
        "args": ["transition", {"duration": 400, "easing": "cubic-in-out"}],
        "initialValue": years[0], "plotlycommand": "animate", "values": years, "visible": True
    }
    fig_dict["layout"]["updatemenus"] = [
        {
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 500, "redraw": False}, "fromcurrent": False, \
                                    "transition": {"duration": 300, "easing": "quadratic-in-out"}}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", \
                                      "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],
            "direction": "left", "pad": {"r": 10, "t": 52}, "showactive": True, "type": "buttons", \
            "x": 0.1, "xanchor": "right", "y": 0, "yanchor": "top"
        }
    ]

    sliders_dict = {
        "active": 0, "yanchor": "top", "xanchor": "left",
        "currentvalue": {
            "font": {"size": 18},
            "prefix": "Year:",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 300, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 15}, "len": 0.9, "x": 0.1, "y": 0, "steps": []
    }

    # make 'fake' data for first year to set up plot correctly
    for country in countries:
        data_dict = {
            "y": list([-999999]),
            "x": list([-999999]),
            "mode": "markers",
            "text": list(['--']),
            "marker": {
                "sizemode": "area",
                "sizeref": 2.5,
                "size": list([1000.0]),
                "color": country_colour(country)
            },
            "name": country
        }
        fig_dict["data"].append(data_dict)

    # make frames for each year of the animation
    for year in years:
        frame = {"data": [], "name": str(year)}
        # ensure elements do not line up between years (this is a hack)
        if (year%2 == 0):
            offset = np.zeros(50)
        else:
            offset = 0

        for country in countries:
            dataset_by_year_and_country = dataset[(dataset['Year'] == int(year)) & (dataset['Country'] == country) & \
                            (dataset['Latitude'] != '--') & (dataset['Longitude'] != '--') & \
                            (dataset['Yield (kT)'] != '--')]
            # set location of each atomic bomb test, the yield and colour for the country
            data_dict = {
                "y": np.insert(dataset_by_year_and_country['Latitude'].values.astype(float), 0, \
                               offset -999999).tolist(),
                "x": np.insert(dataset_by_year_and_country['Longitude'].values.astype(float), 0, \
                               offset -999999).tolist(),
                "mode": "markers",
                "text": np.insert(dataset_by_year_and_country['Date'].values, 0, offset -999999).tolist(),
                "marker": {
                    "sizemode": "area",
                    "sizeref": 2.5,
                    "size": np.fmax(50, np.insert(dataset_by_year_and_country['Yield (kT)'].values.astype(float), \
                                    0, offset + 1000)).tolist(),
                    "color": country_colour(country)
                },
                "name": country
            }
            frame["data"].append(data_dict)

        fig_dict["frames"].append(frame)
        slider_step = {"args": [
            [year],
            {"frame": {"duration": 500, "redraw": False}, "mode": "immediate", "transition": {"duration": 300}}
        ],
            "label": year, "method": "animate"}
        sliders_dict["steps"].append(slider_step)

    fig_dict["layout"]["sliders"] = [sliders_dict]

    # plot the figure
    fig = go.Figure(data=fig_dict) # for scatter plot
    # add image to background
    fig.layout.update(images=[image_layout])

    iplot(fig, config={'scrollzoom': True})

Define the years to be plotted in the animation.

In [ ]:
# create a list of the years to plot in the animation
years = list()
for year in range(1945, 2010):
    years.append(year)

Define the countries to be plotted in the animation.

In [ ]:
# make a list containing just the United States
countries = list(['United States'])

In [ ]:
# make a list of all the countries
countries = list()
for country in dataset['Country']:
    if country not in countries:
        countries.append(country)

Define a function to set the colour of the points for each country. Possible colours recognised by `plotly` (as coded in `python`) include 'darkred', 'crimson', 'red', 'orangered', 'orange', 'yellow', 'green', 'darkgreen', 'blue', 'medium blue', 'navy', 'purple', 'pink', 'grey' and 'black'. The full range of colours can be found at https://matplotlib.org/3.1.0/gallery/color/named_colors.html.

In [ ]:
def country_colour(country):
    # return a pre-defined colour for each input country
    if (country == 'United States'):
        return 'navy'
    elif (country == 'USSR/Russia'):
        return 'crimson'
    elif (country == 'United Kingdom'):
        return 'blue'
    elif (country == 'France'):
        return 'cyan'
    elif (country == 'China'):
        return 'red'
    elif (country == 'India'):
        return 'orange'
    elif (country == 'South Africa'):
        return 'purple'
    elif (country == 'Pakistan'):
        return 'green'
    elif (country == 'North Korea'):
        return 'grey'
    else:
        return 'black'

Create the animation of the location of atomic bomb tests between 1945 and 2009.

In [ ]:
animation(dataset, countries, years)